In [ ]:
import os
import json

import numpy as np

import matplotlib.pyplot as plt

from notebook_helpers import *

In [ ]:
# fig, ax = plt.subplots(1, 3, figsize=(14,2), gridspec_kw={'width_ratios': [1,1,2]})
def plot_as_planes(exp_data, max_steps=1024, cmap_name="plasma"):
    

    my_cmap = plt.get_cmap(cmap_name)
    max_kr = np.max(exp_data[:,1])
    min_kr = np.min(exp_data[:,1])
    
    unique_bits = np.unique(exp_data[:,2])
    unique_bits.sort()

    number_columns = unique_bits.shape[0]

    fig, ax = plt.subplots(1, number_columns+1, \
            figsize=(number_columns*4, 4), gridspec_kw={"width_ratios": [1] * number_columns + [0.25]})

    for ii in range(number_columns):
        plot_data = exp_data[exp_data[:,2] == unique_bits[ii]]
        plot_colors = my_cmap(np.array(plot_data[:,3], dtype=np.uint32))
        ax[ii].scatter(plot_data[:,0], plot_data[:,1], \
                alpha=0.5, \
                c=plot_colors, vmin=0, vmax=max_steps)

        my_title = f"Persistence with {unique_bits[ii]}-bit precision"
        ax[ii].set_title(my_title)
        ax[ii].set_xlabel("time step dt")
        ax[ii].set_ylabel("kernel radius kr")
        ax[ii].set_xticks(np.arange(0.0, 1.1, 0.1))
        ax[ii].set_yticks(np.arange(min_kr, max_kr+int(max_kr/10), int(max_kr/10)))
        
    color_grades = 1000
    my_colors =np.arange(0.0, 1.0, 1/color_grades)[:, None] * np.ones((color_grades, 64))
    
    ax[-1].imshow(my_colors, cmap=cmap_name)
    ax[-1].set_yticks(np.arange(0.0, color_grades+color_grades/4., color_grades/4))
    ax[-1].set_yticklabels(np.arange( .0, 1.0+1/4., 1/4.) * max_steps)
    ax[-1].set_xticklabels("")
    ax[-1].set_xlabel("steps")
    plt.tight_layout()
    
    return fig, ax
    
    
def visualize_results(results_path, exp_idx=None): 
    
    temp_results_dir = os.listdir(results_path)
    

    
    # if a specific results_path is given, skip this part
    exp_folder = None
    
    if exp_idx is not None:
        if "metadata.json" in temp_results_dir:
            exp_folder = results_path
        results_dir = temp_results_dir
    else:
        results_dir = []
        for item  in temp_results_dir:
            if item.startswith("exp"):
                results_dir.append(item)

        results_dir.sort()
        
        for idx, folder in enumerate(results_dir):
            print(f"{idx}      {folder}")

        exp_idx = input(f"please choose an experiment folder (by index number) to visualize. (-1 to visualize all).")
        exp_idx = int(exp_idx)
        
    if exp_idx == -1:

        for idx, folder_name in enumerate(results_dir):
            exp_folder = os.path.join(results_path, folder_name)
            if os.path.isdir(exp_folder):
                metadata_path = os.path.join(exp_folder, "metadata.json")
                if os.path.exists(metadata_path):
                    my_dir_list = os.listdir(exp_folder)
                    for filename in my_dir_list:
                        if filename.startswith("results"):
                            experiment_results_path = os.path.join(exp_folder, filename)
                            my_filename = filename

                            my_data = np.load(experiment_results_path)
                    with open(metadata_path, "r") as f:
                        metadata = json.load(f)

                    fig, ax = plot_as_planes(my_data)
                    fig.suptitle(f"{exp_folder}\n{my_filename[-64:]}", fontsize=22)
                    plt.tight_layout()
                    plt.show()

    elif exp_idx >= 0:
        
        if exp_folder is None:
            exp_folder = os.path.join(results_path, results_dir[exp_idx])
        
        exp_dir = os.listdir(exp_folder)
        metadata_path = os.path.join(exp_folder, "metadata.json")

        for filename in exp_dir:
            if filename.startswith("results"):
                experiment_results_path = os.path.join(exp_folder, filename)
                my_filename = filename
                
        my_data = np.load(experiment_results_path)
        with open(metadata_path, "r") as f:
            metadata = json.load(f)

        fig, ax = plot_as_planes(my_data)

        fig.suptitle(f"{exp_folder}\n{my_filename[-64:]}", fontsize=22)
        plt.tight_layout()
    
        # return figure for more processing
        return fig, ax
    

In [ ]:
default_results_path = os.path.join("..", "results")

In [ ]:
# choose experiment from a printed list
_ = visualize_results(default_results_path)
plt.show()

In [ ]:
# visualize everything
visualize_results(default_results_path, -1)

In [ ]:
# visualize the results found in a specific exp directory
specific_folder = os.path.join(default_results_path, "exp_1687632498404")
_ = visualize_results(specific_folder, 0)

In [ ]:
# visualize the results found in a specific exp directory
specific_folder = os.path.join(default_results_path, "exp_1687632498404")

# visualize_results returns the figure and axes when called with a specific folder
fig, ax = visualize_results(specific_folder, 0)

# the figure can be modified before showing, e.g. with a concise title.
plt.suptitle("Concise Title", fontsize=24)
plt.tight_layout()
plt.show()